<a href="https://colab.research.google.com/github/rmedina0531/data-science/blob/master/data_science_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kobe bryant shot predictor

In [20]:
#import google drive access for data
#unsure if this will work with shared users
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#import libraries 
import pandas as pd
import numpy as np

#import file into data frame from gdrive
data_frame = pd.read_csv('/content/drive/My Drive/datascience/kobe_data.csv')

print(data_frame.head())


         action_type combined_shot_type  ...  opponent  shot_id
0          Jump Shot          Jump Shot  ...       POR        1
1          Jump Shot          Jump Shot  ...       POR        2
2          Jump Shot          Jump Shot  ...       POR        3
3          Jump Shot          Jump Shot  ...       POR        4
4  Driving Dunk Shot               Dunk  ...       POR        5

[5 rows x 25 columns]


In [13]:
#remove all data rows that do not have a shot_made flag
#data is used after the model is train and tested to use for the kaggle competition

#drop the null values in shot_made_flag
data_frame.dropna(subset=['shot_made_flag'], inplace=True)

print(data_frame[['seconds_remaining','shot_made_flag']].head())

   seconds_remaining  shot_made_flag
1                 22             0.0
2                 45             1.0
3                 52             0.0
4                 19             1.0
5                 32             0.0


#Testing using only 2 features using knn

In [16]:
#pull features and labels
#for initial test purposes pulling features: seconds, shot_distance

features = data_frame[['seconds_remaining', 'shot_distance']]
labels = data_frame['shot_made_flag']

print(features.head())

   seconds_remaining  shot_distance
1                 22             15
2                 45             16
3                 52             22
4                 19              0
5                 32             14


In [17]:
#split into training and test data
from sklearn.model_selection import train_test_split


#find out what values to use for test_size and random_state
feature_train, feature_test, label_train, label_test = train_test_split(
    features, labels, test_size=0.25, random_state=6)

print(feature_train.shape)
print(label_train.shape)

(19272, 2)
(19272,)


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#decide on best number of neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(feature_train, label_train)
label_predict = knn.predict(feature_test)

knn_accuracy = accuracy_score(label_test, label_predict)
print(f'KNN accuracy at k=3: {knn_accuracy}')

KNN accuracy at k=3: 0.5400778210116731


#knn using more features
##the knn model will now use
####period
###the following must use hot encoding
action_type,
shot_zone_range,
opponent,
season,
shot_zone_area

In [ ]:
#pull out the features
features = ['action_type','shot_zone_range', ]